#🗺️ **Uso de OpenRouteService en Google Colab**

## En este cuaderno aprenderás a:

1.   Conectarte a la API de OpenRouteService.
2.   Obtener una ruta básica entre dos puntos.
3.   Visualizar esa ruta en un mapa.
4.   Generar una matriz de distancias y tiempos entre múltiples ubicaciones.

# 🔧 **Paso 1: Instalar la librería necesaria**

In [1]:
!pip install openrouteservice folium

# 🔑 **Paso 2: Configurar la API Key**

1.  Crea una cuenta en: https://openrouteservice.org/dev/#/signup  
2.  Obtén tu API key en: https://openrouteservice.org/dev/#/home  
3.  Reemplaza `"TU_API_KEY_AQUI"` por tu propia clave.

In [2]:
import openrouteservice

# Reemplaza con tu propia API key
api_key = "TU_API_KEY_AQUI"

# Crear cliente de OpenRouteService
client = openrouteservice.Client(key=api_key)

# 📍 **Paso 3 (Modificado): Ruta desde un origen, pasando por 6 puntos, y regresando al origen**

En este ejemplo, diseñamos una ruta circular que:

1. Comienza en Bogotá
2. Visita Medellín, Bucaramanga, Cali, Ibagué, Neiva y Villavicencio
3. Regresa a Bogotá

In [13]:
# Lista de coordenadas [longitud, latitud]
coordenadas = [
    [-74.08175, 4.60971],   # Bogotá (origen)
    [-75.56359, 6.25184],   # Medellín
    [-73.1198, 7.1254],     # Bucaramanga
]

# Solicitar la ruta circular
ruta = client.directions(
    coordinates=coordenadas,
    profile='driving-car',
    format='geojson',
    optimize_waypoints=False  # False para respetar el orden dado
)

# Extraer distancia y duración
distancia = ruta['features'][0]['properties']['segments'][0]['distance'] / 1000  # km
duracion = ruta['features'][0]['properties']['segments'][0]['duration'] / 60  # min

print(f"Distancia total: {distancia:.2f} km")
print(f"Duración estimada: {duracion:.2f} minutos")

/usr/local/lib/python3.11/dist-packages/openrouteservice/directions.py:202: UserWarning: Less than 4 coordinates, nothing to optimize!
  warnings.warn("Less than 4 coordinates, nothing to optimize!", UserWarning)


Distancia total: 413.15 km
Duración estimada: 428.47 minutos


# 🗺️ **Paso 4: Visualizar la ruta en un mapa**

In [14]:
import folium

# Crear el mapa centrado en Bogotá
m = folium.Map(location=coordenadas[0][::-1], zoom_start=6)

# Agregar la ruta al mapa
folium.GeoJson(ruta, name="Ruta circular").add_to(m)

# Agregar marcador especial para el origen (Bogotá)
folium.Marker(
    location=coordenadas[0][::-1],
    popup="Origen / Destino: Bogotá",
    icon=folium.Icon(color='green', icon='star', prefix='fa')
).add_to(m)

# Agregar marcadores para los puntos intermedios
for idx, punto in enumerate(coordenadas[1:-1], start=1):  # Ignorar el primer y último punto que es el origen
    folium.Marker(
        location=punto[::-1],
        popup=f"Punto {idx}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)

# Agregar nuevamente marcador final si se desea enfatizar el regreso
folium.Marker(
    location=coordenadas[-1][::-1],
    popup="Regreso: Bogotá",
    icon=folium.Icon(color='darkgreen', icon='flag', prefix='fa')
).add_to(m)

m

# 🧮 **Paso 5: Matriz de Distancias y Tiempos**

Calcularemos las distancias y tiempos entre Bogotá, Medellín, Bucaramanga y Cali.

In [10]:
# Lista de coordenadas
coordenadas = [
    [-74.08175, 4.60971],   # Bogotá (origen)
    [-75.56359, 6.25184],   # Medellín
    [-73.1198, 7.1254],     # Bucaramanga
    [-76.5225, 3.4372],     # Cali
    [-75.233, 4.4389],      # Ibagué
    [-75.28189, 2.93498],   # Neiva
    [-73.6357, 4.142],      # Villavicencio
    [-74.08175, 4.60971]    # Bogotá (regreso)
]

# Nombres para las ciudades
nombres_ciudades = [
    "Bogotá (Inicio)",
    "Medellín",
    "Bucaramanga",
    "Cali",
    "Ibagué",
    "Neiva",
    "Villavicencio",
    "Bogotá (Fin)"
]

# Solicitar matriz de distancias y duración
matriz = client.distance_matrix(
    locations=coordenadas,
    profile='driving-car',
    metrics=['distance', 'duration'],
    units='km'
)

## 📊 **5.1 Visualizar la matriz en tablas con Pandas**

In [12]:
import pandas as pd

# Crear DataFrames con etiquetas personalizadas
df_dist = pd.DataFrame(matriz['distances'], columns=nombres_ciudades, index=nombres_ciudades)
df_time = pd.DataFrame(matriz['durations'], columns=nombres_ciudades, index=nombres_ciudades)

print("🔹 Matriz de Distancias (km):")
display(df_dist.round(2))

print("🔹 Matriz de Tiempos (minutos):")
display(df_time.round(2))

🔹 Matriz de Distancias (km):


,Bogotá (Inicio),Medellín,Bucaramanga,Cali,Ibagué,Neiva,Villavicencio,Bogotá (Fin)
Bogotá (Inicio),0.00,413.15,418.36,451.83,192.11,304.23,111.85,0.00
Medellín,402.81,0.00,388.51,423.46,355.90,590.63,503.19,402.81
Bucaramanga,417.80,387.74,0.00,781.05,512.76,702.27,527.13,417.80
Cali,449.90,428.95,776.43,0.00,262.45,362.69,545.03,449.90
Ibagué,194.01,404.65,511.57,264.83,0.00,210.40,289.14,194.01
Neiva,305.91,593.67,700.59,362.99,209.93,0.00,401.04,305.91
Villavicencio,110.63,518.50,528.16,547.23,287.50,399.62,0.00,110.63
Bogotá (Fin),0.00,413.15,418.36,451.83,192.11,304.23,111.85,0.00


🔹 Matriz de Tiempos (minutos):


,Bogotá (Inicio),Medellín,Bucaramanga,Cali,Ibagué,Neiva,Villavicencio,Bogotá (Fin)
Bogotá (Inicio),0.00,25708.09,25229.39,29367.32,12759.68,17165.51,5585.22,0.00
Medellín,26085.84,0.00,18577.12,25292.49,22326.24,31178.81,30910.14,26085.84
Bucaramanga,25986.47,17850.87,0.00,40340.00,23471.36,32202.22,31143.94,25986.47
Cali,28258.92,24533.66,39117.56,0.00,15916.89,25568.58,32117.04,28258.92
Ibagué,13039.09,22360.16,23618.90,17284.52,0.00,10353.29,16897.20,13039.09
Neiva,17268.04,31069.91,32328.64,25573.92,10406.63,0.00,21126.16,17268.04
Villavicencio,5754.95,30722.62,30539.82,33757.53,17149.90,21555.72,0.00,5754.95
Bogotá (Fin),0.00,25708.09,25229.39,29367.32,12759.68,17165.51,5585.22,0.00
